In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, ReLU, Flatten, LSTM, Input
from tensorflow.keras.layers import Input, Embedding, BatchNormalization, Concatenate
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.models import Sequential, Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from keras.utils import to_categorical
import re
from sklearn.model_selection import train_test_split

Lấy data từ 2 file X_train Y_train X_test và Y_test từ các file X_train_cleaned Y_train_cleaned X_test_cleaned Y_test_cleaned

In [4]:
X_tr=pd.read_csv('Twitter_2/X_train_cleaned.csv',header=None,encoding='latin1')
Y_tr=pd.read_csv('Twitter_2/Y_train_cleaned.csv',header=None)
X_t=pd.read_csv('Twitter_2/X_test_cleaned.csv',header=None,encoding='latin1')
Y_t=pd.read_csv('Twitter_2/Y_test_cleaned.csv',header=None)
X_train = X_tr[0].astype(str).tolist()
X_test = X_t[0].astype(str).tolist()


Chuẩn bị data để nhét vào mô hình chuẩn bị việc train mô hình

In [5]:
encoder=LabelEncoder()
Y_train=encoder.fit_transform(Y_tr)
Y_test=encoder.transform(Y_t)
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
sequences=tokenizer.texts_to_sequences(X_train)
tr_x=pad_sequences(sequences,maxlen=50)
tr_y=to_categorical(Y_train)
sequences=tokenizer.texts_to_sequences(X_test)
t_x=pad_sequences(sequences,maxlen=50)
t_y=to_categorical(Y_test)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Tạo mô hình CNN để chạy data

In [20]:
embedding_dim=32
max_len=50
max_word=20000
batchsize=64
epochs=25
def model1():
    inp=Input(shape=(max_len,))
    
    embed=Embedding(input_dim=max_word,output_dim=embedding_dim,input_length=max_len)(inp)
    conv=Conv1D(128,3,padding='same',activation='relu')(embed)
    batch=BatchNormalization()(conv)
    glob=GlobalMaxPooling1D()(batch)
    drop=Dropout(0.5)(glob)
    outp=Dense(3,activation='softmax')(drop)

    model=Model(inputs=inp,outputs=outp)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',Precision,Recall])
    return model

Chạy mô hình với data train

In [21]:
model=model1()
model.summary()
history=model.fit(tr_x, tr_y, epochs=epochs, batch_size=batchsize, validation_split=0.2)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (None, 50, 32)         │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 50, 128)        │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 50, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_5          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 653,315 (2.49 MB)

 Trainable params: 653,059 (2.49 MB)

 Non-trainable params: 256 (1.00 KB)

Epoch 1/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.4802 - loss: 0.6233 - precision_4: 0.5115 - recall_4: 0.3659 - val_accuracy: 0.6251 - val_loss: 0.5751 - val_precision_4: 0.8009 - val_recall_4: 0.3096
Epoch 2/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7416 - loss: 0.4129 - precision_4: 0.7560 - recall_4: 0.7206 - val_accuracy: 0.7077 - val_loss: 0.4385 - val_precision_4: 0.7216 - val_recall_4: 0.6889
Epoch 3/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8204 - loss: 0.3200 - precision_4: 0.8306 - recall_4: 0.8104 - val_accuracy: 0.7067 - val_loss: 0.4542 - val_precision_4: 0.7142 - val_recall_4: 0.6946
Epoch 4/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8761 - loss: 0.2360 - precision_4: 0.8820 - recall_4: 0.8710 - val_accuracy: 0.6822 - val_loss: 0.5158 - val_precision_4: 0.6920 - val_recall_4: 0.6740
Epoch 5/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9127 - loss: 0.1819 - precision_4: 0.9173 - recall_4: 0.9089 -

Thử dữ liệu với data test

In [22]:
pred_label=model.predict(t_x)
predicted=np.argmax(pred_label,axis=1)
true=np.argmax(t_y,axis=1)
print(classification_report(true, predicted))


111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.61      0.59      0.60      1001
           1       0.59      0.62      0.61      1430
           2       0.68      0.67      0.67      1103

    accuracy                           0.62      3534
   macro avg       0.63      0.62      0.63      3534
weighted avg       0.63      0.62      0.62      3534

